<a href="https://www.kaggle.com/code/raneemabdo/gradient-boosting?scriptVersionId=144785100" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [ ]:
bikes=pd.read_csv("https://raw.githubusercontent.com/DrSaadLa/PythonTuts/main/ML_and_DL%20Workshop/bike_rentals.csv")

In [ ]:
bikes.info()

In [ ]:
bikes.head()

In [ ]:
bikes.describe().T

In [ ]:
bikes.isnull().sum()

In [ ]:
bikes['windspeed'].fillna((bikes['windspeed'].median()),inplace=True)

In [ ]:
bikes['hum']=bikes['hum'].fillna(bikes.groupby('season')['hum'].transform('median'))

In [ ]:
mean_temp=(bikes.iloc[700]['temp']+bikes.iloc[702]['temp'])/2
mean_atemp=(bikes.iloc[700]['atemp']+bikes.iloc[702]['atemp'])/2
bikes['temp'].fillna((mean_temp),inplace=True)
bikes['atemp'].fillna((mean_atemp),inplace=True)

In [ ]:
bikes['dteday']=bikes['dteday'].apply(pd.to_datetime,errors='coerce')

In [ ]:
import datetime as dt
bikes['mnth']=bikes['dteday'].dt.month
bikes.loc[730,'yr']=1.0
bikes=bikes.drop('dteday',axis=1)
bikes=bikes.drop(['casual','registered'],axis=1)

In [ ]:
x=bikes.iloc[:,:-1]
y=bikes.iloc[:,-1]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=1)

In [ ]:
lin_reg=LinearRegression()
lin_reg.fit(x_train,y_train)
pred=lin_reg.predict(x_test)
mse=MSE(y_test,pred)
rmse=np.sqrt(mse)
print("RMSE",rmse)

In [ ]:
tree_reg=DecisionTreeRegressor(max_depth=2,random_state=1)
tree_reg.fit(x_train,y_train)
y_pred=tree_reg.predict(x_test)
mse=MSE(y_test,y_pred)
rmse=np.sqrt(mse)
print("RMSE",rmse)

**phase one on gradient boosting    manual**

In [ ]:
treereg_1=DecisionTreeRegressor(max_depth=2,random_state=1)
treereg_1.fit(x_train,y_train)
ytrain_pred=treereg_1.predict(x_train)
errors_1=y_train-ytrain_pred
print(treereg_1)

**phase two**

In [ ]:
treereg_2=DecisionTreeRegressor(max_depth=2,random_state=1)
treereg_2.fit(x_train,errors_1)
y2train_pred=treereg_2.predict(x_train)
errors_2=errors_1-y2train_pred
print(treereg_2)

**phase three**

In [ ]:
treereg_3=DecisionTreeRegressor(max_depth=2,random_state=1)
treereg_3.fit(x_train,errors_2)
print(treereg_3)

**final step**

In [ ]:
first_pred=treereg_1.predict(x_test)
second_pred=treereg_2.predict(x_test)
third_pred=treereg_3.predict(x_test)
final_pred=first_pred+second_pred+third_pred

In [ ]:
final_rmse=np.sqrt(MSE(y_test,final_pred))
print("the rmse achived by three tree estimators",final_rmse)

In [ ]:
gbreg=GradientBoostingRegressor(max_depth=2,n_estimators=3,random_state=1,learning_rate=1.0)
gbreg.fit(x_train,y_train)
y_pred=gbreg.predict(x_test)
print("rmse achived by three tree estimators",(np.sqrt(MSE(y_test,y_pred))))